# Clickbait Detection: SVM (No Scaling) & UMAP Embeddings

## Εισαγωγή
Στο παρόν notebook διερευνούμε την απόδοση ενός μοντέλου **Support Vector Machine (SVM)** με πυρήνα RBF, εφαρμοσμένο απευθείας στα "ωμά" UMAP embeddings.

### Το Θεωρητικό Υπόβαθρο (The Scaling Paradox)
Στη θεωρία της Μηχανικής Μάθησης, το SVM θεωρείται "distance-based" αλγόριθμος και συνήθως απαιτεί κανονικοποίηση (StandardScaler) για να λειτουργήσει σωστά.
Ωστόσο, τα πειράματά μας έδειξαν ότι στα συγκεκριμένα δεδομένα (**BERT embeddings μειωμένα με UMAP**), η κανονικοποίηση λειτουργεί **καταστροφικά**, ρίχνοντας την απόδοση κατά ~15%.

**Γιατί συμβαίνει αυτό;**
Ο αλγόριθμος UMAP κατασκευάζει έναν τοπολογικό χώρο όπου η **πυκνότητα** και οι **αποστάσεις** έχουν φυσική σημασία. Εφαρμόζοντας `StandardScaler`, "βιάζουμε" όλες τις διαστάσεις να έχουν την ίδια διασπορά, ενισχύοντας τον θόρυβο στις δευτερεύουσες διαστάσεις και αλλοιώνοντας τη γεωμετρία που έμαθε το UMAP.

Επομένως, σε αυτό το πείραμα **δεν εφαρμόζουμε scaling**, επιτρέποντας στο SVM να βρει τα διαχωριστικά υπερεπίπεδα στον φυσικό χώρο του UMAP.

## Στόχοι
1. **Φόρτωση Δεδομένων**: Χρήση των UMAP-500 parquets χωρίς περαιτέρω επεξεργασία.
2. **Βελτιστοποίηση (Optuna)**: Εύρεση των βέλτιστων `C` (ποινή σφάλματος) και `kernel` παραμέτρων.
3. **Champion Model**: Εκπαίδευση του τελικού μοντέλου χωρίς περιορισμούς επαναλήψεων.

In [ ]:
import pandas as pd
import numpy as np
import optuna
import mlflow
import sys
import os
import time
import warnings

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# Απόκρυψη FutureWarnings για καθαρό output
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

# Ρύθμιση Paths
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import mlflow_helper

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
DATA_FOLDER = os.path.join(project_root, 'data', 'clean', 'umap')

print(f"Project Root: {project_root}")
print(f"Data Folder: {DATA_FOLDER}")

## Φόρτωση Δεδομένων

Φορτώνουμε τα Train/Valid/Test sets.
**Προσοχή:** Εδώ, σε αντίθεση με τα κλασικά pipelines του SVM, **δεν** θα δημιουργήσουμε Scaler. Τα δεδομένα θα περάσουν στο μοντέλο ακριβώς όπως αποθηκεύτηκαν από το UMAP.

In [ ]:
def load_split_data(data_path):
    files = {
        "Train": "train_umap_500.parquet",
        "Valid": "valid_umap_500.parquet",
        "Test":  "test_umap_500.parquet"
    }

    loaded_data = {}
    possible_label_cols = ['labels', 'label', 'target', 'class', 'is_clickbait']

    print(f"⏳ Έναρξη φόρτωσης από: {data_path}")

    for name, filename in files.items():
        file_path = os.path.join(data_path, filename)
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Το αρχείο {filename} δεν βρέθηκε.")

        try:
            df = pd.read_parquet(file_path, engine='fastparquet')
        except:
            df = pd.read_parquet(file_path, engine='pyarrow')

        # 1. Εντοπισμός Features
        feature_cols = [c for c in df.columns if c.startswith("umap_")]
        if not feature_cols:
            feature_cols = [c for c in df.columns if c not in possible_label_cols]

        # 2. Εντοπισμός Label
        label_col = None
        for col in possible_label_cols:
            if col in df.columns:
                label_col = col
                break

        if label_col is None:
             raise ValueError(f"Δεν βρέθηκε label στήλη στο {filename}")

        if label_col in feature_cols:
            feature_cols.remove(label_col)

        y = df[label_col].values.astype(int)
        X = df[feature_cols].values.astype(np.float32)

        loaded_data[name] = (X, y)
        print(f"   ✅ [{name}] Loaded: X={X.shape}, y={y.shape}")

    return loaded_data["Train"], loaded_data["Valid"], loaded_data["Test"]

# Εκτέλεση Φόρτωσης
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_split_data(DATA_FOLDER)

## Βελτιστοποίηση Υπερπαραμέτρων (Tuning)

Ορίζουμε τη συνάρτηση `objective` για το Optuna.

**Σημαντικές Λεπτομέρειες Υλοποίησης:**
1. **Περιορισμός `max_iter=2000`:** Το SVM χωρίς scaling μπορεί να δυσκολευτεί πολύ να συγκλίνει (να βρει τη βέλτιστη λύση). Αν το αφήσουμε χωρίς όριο, ένα μόνο trial μπορεί να πάρει ώρες. Βάζουμε όριο 2000 επαναλήψεων για να ολοκληρωθεί το tuning σε λογικό χρόνο.
2. **Search Space:**
   * `kernel`: Δοκιμάζουμε `linear` και `rbf`.
   * `C`: Ελέγχει την ποινή σφάλματος. Μικρό C = απλό μοντέλο (underfitting), Μεγάλο C = αυστηρό μοντέλο (overfitting).
   * `gamma`: Για τον RBF kernel, καθορίζει πόσο "μακριά" φτάνει η επιρροή κάθε δείγματος.

In [ ]:
def objective(trial, X_tr, y_tr, X_v, y_v):
    # --- 1. Hyperparameters Space ---
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf"])
    C = trial.suggest_float("C", 1e-3, 100.0, log=True)

    # Ρυθμίσεις για ταχύτητα κατά το Tuning
    params = {
        "kernel": kernel,
        "C": C,
        "probability": True, # Χρήσιμο για μετρικές όπως ROC-AUC, αλλά βαραίνει την εκπαίδευση
        "random_state": 42,
        "max_iter": 2000 # Αυστηρό όριο για να μην κολλήσει το tuning
    }

    if kernel == "rbf":
        params["gamma"] = trial.suggest_categorical("gamma", ["scale", "auto"])

    # --- 2. Model Setup (NO SCALING) ---
    model = SVC(**params)

    # --- 3. Training ---
    t0 = time.time()
    model.fit(X_tr, y_tr)
    training_time = time.time() - t0

    # --- 4. Validation ---
    preds = model.predict(X_v)
    f1 = f1_score(y_v, preds)
    acc = accuracy_score(y_v, preds)

    metrics = {"val_f1": f1, "val_accuracy": acc, "training_time_sec": training_time}

    # Καταγραφή παραμέτρου "No Scaling"
    params_to_log = params.copy()
    params_to_log["scaler"] = "None"

    mlflow_helper.log_optuna_trial(trial, params_to_log, metrics, model, run_name_prefix="SVM_Trial")

    return f1

## Εκτέλεση Πειράματος & Champion Model

**Βήμα 1: Tuning**
Τρέχουμε το Optuna για **10 trials**.
* *Γιατί μόνο 10;* Το SVM έχει πολυπλοκότητα $O(n^2)$ έως $O(n^3)$. Με 85.000 δείγματα εκπαίδευσης και 500 διαστάσεις, είναι εξαιρετικά αργό. Τα 10 trials είναι αρκετά για να πάρουμε μια καλή ένδειξη των παραμέτρων χωρίς να περιμένουμε μέρες.

**Βήμα 2: Champion Model Training**
Εκπαιδεύουμε το τελικό μοντέλο με τις βέλτιστες παραμέτρους.
* **Σημαντικό:** Εδώ θέτουμε `max_iter = -1` (χωρίς όριο). Στο τελικό μοντέλο θέλουμε να βρει την απόλυτα βέλτιστη λύση, ακόμα κι αν χρειαστεί περισσότερη ώρα για να συγκλίνει.

In [ ]:
EXPERIMENT_NAME = "Clickbait_SVM_UMAP_NoScaling"
mlflow_helper.setup_mlflow(EXPERIMENT_NAME)

print(f"\n🚀 Έναρξη Πειράματος: {EXPERIMENT_NAME}")

# --- ΦΑΣΗ 1: Tuning ---
print("\n🔍 [ΦΑΣΗ 1] Hyperparameter Tuning (Optuna)...")
with mlflow.start_run(run_name="SVM_Hyperparameter_Tuning"):
    mlflow.log_param("dataset", "UMAP_500")
    mlflow.log_param("scaling", "None")

    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", sampler=sampler)

    # 10 trials λόγω του υπολογιστικού κόστους
    study.optimize(
        lambda t: objective(t, X_train, y_train, X_val, y_val),
        n_trials=10
    )

    print("🏆 Best F1:", study.best_value)
    print("🏆 Best Params:", study.best_params)


# --- ΦΑΣΗ 2: Champion Model ---
print("\n👑 [ΦΑΣΗ 2] Champion Model Training...")
with mlflow.start_run(run_name="SVM_Champion_Model") as final_run:
    best_params = study.best_params

    # Ενημέρωση παραμέτρων για το τελικό μοντέλο
    best_params.update({
        "probability": True,
        "random_state": 42,
        "max_iter": -1  # Αφήνουμε το μοντέλο να τρέξει μέχρι να συγκλίνει πλήρως
    })

    mlflow.log_params(best_params)
    mlflow.log_param("dataset", "UMAP_500_NoScaling")

    # 1. Προετοιμασία Δεδομένων (Full Train - No Scaling)
    X_full = np.concatenate((X_train, X_val))
    y_full = np.concatenate((y_train, y_val))

    final_model = SVC(**best_params)

    # 2. Training
    # Αυτό μπορεί να πάρει ώρα, καθώς το SVM χωρίς scaling αργεί να συγκλίνει
    print("⏳ Εκπαίδευση τελικού μοντέλου (αυτό μπορεί να καθυστερήσει)...")
    start_t = time.time()
    final_model.fit(X_full, y_full)
    final_train_time = time.time() - start_t
    print(f"⏱️ Training Time: {final_train_time:.2f} sec")

    # 3. Logging
    mlflow.sklearn.log_model(final_model, artifact_path="champion_model")

    # 4. Evaluation
    print("📈 Αξιολόγηση στο Test Set...")
    mlflow_helper.evaluate_and_log_metrics(
        final_model,
        X_test, # Raw data
        y_test,
        prefix="test",
        training_time=final_train_time
    )

    print(f"\n✅ Ολοκληρώθηκε. Run ID: {final_run.info.run_id}")